<a href="https://colab.research.google.com/github/susej2/AlgoritmosOptimizacion/blob/main/Algoritmos_R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alumno: Jesus Martinez <br>
Github: https://github.com/susej2/AlgoritmosOptimizacion/
<br> **Descripcion del reto:** Mejorar la implementación de búsqueda local implementada en  clase sobre el TSP con otros operadores de vecindad.

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.20.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios
import numpy as np

#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)




gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())

distancias = np.zeros((len(Nodos), len(Nodos)))

for i in range(len(Nodos)):
    for j in range(len(Nodos)):
        distancias[i][j] = problem.get_weight(Nodos[i], Nodos[j])



In [ ]:
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion

def distancia_total(solucion, distancias):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancias[solucion[i]][solucion[i + 1]]
    distancia_total += distancias[solucion[-1]][solucion[0]]
    return distancia_total

def buscar_solucion_tabu(distancias, tamano_lista_tabu, num_iteraciones):
    num_ciudades = len(distancias)
    mejor_solucion = crear_solucion(Nodos)
    mejor_distancia = distancia_total(mejor_solucion, distancias)
    solucion_actual = mejor_solucion.copy()

    lista_tabu = []

    for iteracion in range(num_iteraciones):
        vecina_encontrada = False
        mejor_vecina = None
        mejor_distancia_vecina = float('inf')

        for i in range(1, num_ciudades - 1):
            for j in range(i + 1, num_ciudades):
                vecina = solucion_actual.copy()
                vecina[i], vecina[j] = vecina[j], vecina[i]

                distancia_vecina = distancia_total(vecina, distancias)

                if distancia_vecina < mejor_distancia_vecina and (i, j) not in lista_tabu:
                    mejor_vecina = vecina
                    mejor_distancia_vecina = distancia_vecina
                    vecina_encontrada = True

        if vecina_encontrada:
            solucion_actual = mejor_vecina
            lista_tabu.append((i, j))
            if len(lista_tabu) > tamano_lista_tabu:
                lista_tabu.pop(0)

        if mejor_distancia_vecina < mejor_distancia:
            mejor_solucion = mejor_vecina
            mejor_distancia = mejor_distancia_vecina

    return mejor_solucion, mejor_distancia

# Ejemplo de uso
solucion_optima, distancia_optima = buscar_solucion_tabu(distancias, tamano_lista_tabu=5, num_iteraciones=1000)

print("Mejor solución encontrada:", solucion_optima)
print("Distancia óptima:", distancia_optima)


Mejor solución encontrada: [0, 27, 2, 28, 9, 23, 41, 18, 26, 5, 6, 4, 39, 21, 24, 40, 19, 13, 12, 11, 25, 10, 8, 29, 30, 22, 38, 34, 32, 3, 1, 7, 35, 36, 37, 15, 16, 14, 17, 31, 20, 33]
Distancia óptima: 1930.0
